In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import logging
import gzip
import gensim 
import re
import spacy
import math
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tqdm import tqdm


In [ ]:
print(torch.cuda.is_available())
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

True


'cuda:0'

# Preprocessing & Loading data

## Preprocess 

### Clothing Review Dataset

In [ ]:
# Load dataset for clothing reviews
reviews = pd.read_csv("/content/drive/MyDrive/Comp550/reviews.csv")
reviews = reviews.dropna()
print(reviews.shape)
reviews.head()

(19662, 11)


,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
5,5,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses
6,6,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits


In [ ]:
reviews['Title'] = reviews['Title'].fillna('')
reviews['Review Text'] = reviews['Review Text'].fillna('')
reviews['review'] = reviews['Title'] + ' ' + reviews['Review Text']

In [ ]:
reviews = reviews[['review', 'Rating']]
reviews.columns = ['review', 'rating']
reviews.head()

,review,rating
2,Some major design flaws I had such high hopes ...,3
3,"My favorite buy! I love, love, love this jumps...",5
4,Flattering shirt This shirt is very flattering...,5
5,Not for the very petite I love tracy reese dre...,2
6,Cagrcoal shimmer fun I aded this in my basket ...,5


#### Tokenize each sentence

In [ ]:
#take advantage of nltk to tokenize all sentences
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
en_stop_words = set(stopwords.words('english'))
tokenizer = nltk.RegexpTokenizer(r'\w+|\$[\d\.]+')

def tokenize_sent(sent):
    
    tokenized = tokenizer.tokenize(sent)
    filtered = [w.lower() for w in tokenized if w.lower() not in en_stop_words]
    return filtered

reviews['tokenized'] = reviews['review'].apply(lambda x: tokenize_sent(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Sanity cehck for tokenizers
print(reviews.head(3))

                                              review  ...                                          tokenized
2  Some major design flaws I had such high hopes ...  ...  [major, design, flaws, high, hopes, dress, rea...
3  My favorite buy! I love, love, love this jumps...  ...  [favorite, buy, love, love, love, jumpsuit, fu...
4  Flattering shirt This shirt is very flattering...  ...  [flattering, shirt, shirt, flattering, due, ad...

[3 rows x 3 columns]


#### Encode and truncate sentence

In [ ]:
# get unique words in the corpus
all_words = []
for x in reviews['tokenized']:
    all_words.extend(x)

word_set = list(set(all_words))
word_count = Counter(all_words)

# filter out words with low frequency
for word_list in reviews.tokenized:
    new_list = []
    for word in word_list:
        if word_count[word] > 2:
            new_list.append(word)
    word_list = new_list

# update set of words after removing the ones with low frequency
new_word_list = []
for x in reviews['tokenized']:
    new_word_list.extend(x)
word_set = list(set(new_word_list))

# map each unique words & unknown token in reviews.encoded to an index
word2index = {}
word2index['<UNK>'] = 0
word2index['<PAD>'] = 1

for i, word in enumerate(word_set, 2):
    word2index[word] = i

# encode the original sequence
def encode(sent_list):
    result = []
    for x in sent_list:
        index = word2index[x]
        result.append(index)
    return result

reviews['encoded'] = reviews.tokenized.apply(lambda x: encode(x))

print(reviews.head())

# get sequence average length
total_len = 0
for x in reviews.encoded:
    total_len += len(x)

ave_len = math.floor(total_len/reviews.shape[0])

# filter out long sequences --> encode all sequence to length = ave_len
# pad short sequence

for i, row in reviews.iterrows():
    size = min(len(row.encoded), ave_len)
    new_encoded = row.encoded[:size]
    if size < ave_len:
        for j in range(0, ave_len - len(row.encoded)):
            new_encoded.append(1)
    reviews.at[i, 'encoded'] = new_encoded

reviews['review_length'] = reviews.encoded.apply(lambda x: len(x))

                                              review  ...                                            encoded
2  Some major design flaws I had such high hopes ...  ...  [11980, 2991, 10896, 13138, 5522, 10712, 10427...
3  My favorite buy! I love, love, love this jumps...  ...  [10243, 7421, 6018, 6018, 6018, 84, 12961, 455...
4  Flattering shirt This shirt is very flattering...  ...  [8274, 11783, 11783, 8274, 12037, 10972, 9032,...
5  Not for the very petite I love tracy reese dre...  ...  [412, 6018, 7035, 9547, 9160, 1444, 412, 11017...
6  Cagrcoal shimmer fun I aded this in my basket ...  ...  [6431, 9653, 12961, 1806, 10583, 10271, 288, 2...

[5 rows x 4 columns]


In [ ]:
# sanity check for encoding:
print(len(all_words))
print(ave_len)
lengths = [len(x) for x in reviews.encoded]
print(max(lengths))
print(min(lengths))
print(set(reviews.rating))
review_length = reviews.review_length
reviews.head()
print(len(word_set))

639434
32
32
32
{1, 2, 3, 4, 5}
14088


In [ ]:
# Train test split from skearln
data_size = len(reviews['encoded'])
assert data_size == len(reviews['rating']) 
X, y = list(zip(list(reviews['encoded']),(list(reviews['review_length'])))), list(reviews['rating'])
# X(data, length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

In [ ]:
# Sanity check for length match
print(X_train[0])

([7433, 5773, 8514, 6811, 2174, 5773, 11272, 3030, 7421, 10213, 7433, 6811, 9674, 13429, 5478, 6018, 11017, 2074, 4810, 6296, 13583, 10213, 2193, 11472, 6811, 9674, 9879, 9607, 1860, 2193, 4304, 9674], 32)


## Dataset and Dataloader

### Clothing Review Dataset

In [ ]:
class Dataset_Glove(Dataset):
    def __init__(self, X, y): 
        self.X = X
        y_new = [i-1 for i in y]
        self.y = y_new

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return  self.X[idx], self.y[idx]

In [ ]:
trainSet_glv = Dataset_Glove(X_train, y_train)
testSet_glv = Dataset_Glove(X_test, y_test)

In [ ]:
print(trainSet_glv[0])

(([7433, 5773, 8514, 6811, 2174, 5773, 11272, 3030, 7421, 10213, 7433, 6811, 9674, 13429, 5478, 6018, 11017, 2074, 4810, 6296, 13583, 10213, 2193, 11472, 6811, 9674, 9879, 9607, 1860, 2193, 4304, 9674], 32), 4)


In [ ]:
def myCollate(batch):
    data = [item[0] for item in batch]
    label = [item[1] for item in batch]
    # sort sequence according to it's length in a batch
    data = data
    # data.sort(key=lambda x: x[1], reverse = True)

    # data (review: list, length: int)
    review = torch.tensor([x[0] for x in data], dtype = torch.long)
    label = torch.tensor(label, dtype = torch.long)
    seq_len = [x[1] for x in data]
    return review, label, seq_len

In [ ]:
trainSet_glv = Dataset_Glove(X_train, y_train)
testSet_glv = Dataset_Glove(X_test, y_test)

In [ ]:
trainLoader_glv = DataLoader(dataset = trainSet_glv, batch_size = 16, collate_fn = myCollate, shuffle = True, drop_last = True)
testLoader_glv = DataLoader(dataset = testSet_glv, batch_size = 16, collate_fn = myCollate, shuffle = True, drop_last = True)
it = iter(trainLoader_glv)
x, y, seq_len = it.next()

In [ ]:
x.data

tensor([[   62, 10712,  7973, 10712,  3324,  2911,  8810,  1732,  8432, 11815,
          5552, 10712,  6018,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [10712,  5798,  9855,  9266, 10712,  3966,  3173,  9857,  9169,  9855,
         12153,  7755,  2542,  9890,   614, 11859, 12277, 10573, 13178, 12048,
         10712, 13606,  7552,  9635, 10929, 13606,   241, 13799, 10929,  8834,
          6814, 10929],
        [ 1130,    54, 13904, 11017,  2743,  5139, 12712,   412,  3300,  2946,
         10712, 13113, 11783,  7860,  3966,  6881, 13280,  8013,  1130,  9261,
          1022,  6231,  8950,  5798, 11537,  2954,  7337,  6231, 10511,  8363,
         13799, 10712],
        [ 6018,   614, 11859,  7949,  3375,  3665,  6814,  7949,  5168, 11054,
         13178, 10218,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,  

In [ ]:
y

tensor([4, 4, 3, 4, 4, 4, 3, 4, 4, 3, 4, 3, 4, 4, 4, 4])

In [ ]:
seq_len

[32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32]

In [ ]:
class LSTM_fixed_len(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 5)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, l):
        x = self.embedding(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

## Embedding

In [ ]:
def validation_metrics(model, valid_dl):
    print("current in vlaidaiton")
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    criterion = nn.CrossEntropyLoss()
    model = model.to(device)
    y_total = []
    y_pred_total = []
    for x, y, seq_len in valid_dl:
        x = x.long().to(device)
        y = y.long().to(device)
        y_hat = model(x, seq_len)
        loss = criterion(y_hat, y)
        
        # print(f"shape of y: {y.shape}")
        # print(f"shape of y_pred: {y_hat.shape}")
        # print(y_hat)
        pred = torch.argmax(y_hat, 1)
        y_total.extend(y.tolist())
        y_pred_total.extend(pred.tolist())
        
        # print(pred)
        correct += sum((pred == y))
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        # print(len(y_total))
        # print(len(y_pred_total))

      # calculate precision, recall, and f1
    f1 = (f1_score(y_total, y_pred_total, average='weighted'))
    precision = (precision_score(y_total, y_pred_total, average='weighted'))
    recall =(recall_score(y_total, y_pred_total, average='weighted'))
    return sum_loss/total, correct/total, precision, recall, f1

In [ ]:
def train_model(model, fname, epochs = 50, lr = 0.0001):
    parameters = filter(lambda p:p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr = lr)
    criterion = nn.CrossEntropyLoss()
    model = model.to(device)
    PATH = '/content/drive/MyDrive/McGill/Comp 550/' + fname
    epoch = 0
    best_val_acc = 0.0
    for i in range(epochs):
        epoch += 1
        print(f"At epoch {i}")
        batch_count = 0
        model.train()
        sum_loss = 0.0
        total = 0.0
        train_correct = 0.0
        for x, y, seq_len in tqdm(trainLoader_glv):
            x = x.long().to(device)
            y = y.long().to(device)
            batch_count += 1
            optimizer.zero_grad()
            y_pred = model(x, seq_len)
            pred = torch.argmax(y_pred, 1)
            train_correct += sum((pred == y))
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc, precision, recall, f1 = validation_metrics(model, testLoader_glv)



        print("train loss %.3f, train accuracy %.3f, val loss %.3f, val accuracy %.3f, precision %.3f, recall %.3f, F1 %.3f" 
              % (sum_loss/total, train_correct/total, val_loss, val_acc, precision, recall, f1))
        if val_acc > best_val_acc and i>=2:
            best_val_acc = val_acc
            NEW_PATH = PATH+f'_{i}_{val_acc}'
            #torch.save(model.state_dict(), NEW_PATH)
            print(f"\t=> Best model saved at {i}th epoch with valication accuracy of {val_acc}")

In [ ]:
vanilla_LSTM = LSTM_fixed_len(len(all_words), 50, 50)

In [ ]:
train_model(vanilla_LSTM,"vanilla_lstm")

At epoch 0


100%|██████████| 983/983 [00:08<00:00, 113.80it/s]


current in vlaidaiton


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


train loss 1.308, train accuracy 0.507, val loss 1.215, val accuracy 0.553, precision 0.306, recall 0.553, F1 0.394
At epoch 1


...

current in vlaidaiton
train loss 0.740, train accuracy 0.685, val loss 1.083, val accuracy 0.620, precision 0.545, recall 0.620, F1 0.558
At epoch 48


100%|██████████| 983/983 [00:08<00:00, 114.88it/s]


current in vlaidaiton
train loss 0.734, train accuracy 0.686, val loss 1.097, val accuracy 0.621, precision 0.546, recall 0.621, F1 0.558
At epoch 49


100%|██████████| 983/983 [00:08<00:00, 115.21it/s]


current in vlaidaiton
train loss 0.727, train accuracy 0.690, val loss 1.069, val accuracy 0.622, precision 0.553, recall 0.622, F1 0.567
